## Cleaning the data scraped from indeed.com

#### Steps taken:
1. add company-wide columns that pick bar pct for:
       Quick-paced
       Slow-paced
       Stressful
       Balanced
       Competitive
       Relaxed
       Cut-throat
2. add company-wide columns for ratings:
    Compensation/Benefits
    Culture
    Job Security
    Work/Life Balance
    Management

3. parse "width: 80%" to 1 - 5 and NaN for 0%
    b. add names to identify review-specific ratings

#### TO DO:

Load file

5. parse currently or not currently employed

6. analyze and remove duplicates 

4. parse company-wide attributes
    company_count_photos	
    company_count_reviews	
    company_count_salaries	
    company_empl	
    company_facebook	
    company_hq	
    company_indeed_url
   

In [53]:
import numpy as np
import pandas as pd
import re

In [54]:
file = "indeed_scrapy/scrape_data/company_reviews100to500fort20171020.csv"
df = pd.read_csv(file)


In [55]:
print(df.shape)
df.head()


(507557, 54)


,agg_rating,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,bar_pct1,bar_pct2,bar_pct3,bar_pct4,...,rating_scr3,rating_scr4,rating_scr5,review_date,review_title,reviewer_company_empl_status,reviewer_job_location,reviewer_job_title,top_list,work_life_rating
0,width: 100.0%;,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,...,3.1,3.1,3.0,"April 18, 2012",Many Opportunities in a Fast Growing Company,(Current Employee) –,"Victor, NY",Plant Process Information Manager,Fortune 500 2017-03-31,width: 100.0%;
1,width: 80.0%;,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,...,3.1,3.1,3.0,"April 15, 2012","good team mates,and friends",(Former Employee) –,"tri-state area,Nevada,California,Arizona",senior service speicalest,Fortune 500 2017-03-31,width: 60.0%;
2,width: 100.0%;,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,...,3.1,3.1,3.0,"April 13, 2012",Challenging and educational workplace,(Current Employee) –,"Saint Paul, MN",Route Sales Manager,Fortune 500 2017-03-31,width: 0.0%;
3,width: 100.0%;,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,...,3.1,3.1,3.0,"April 7, 2012",Ideal professional career position and company...,(Former Employee) –,"Eagan, MN",Collections Specialist/ Call Center,Fortune 500 2017-03-31,width: 100.0%;
4,width: 60.0%;,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,...,3.1,3.1,3.0,"April 3, 2012",Good company,(Former Employee) –,"Orlando, FL",Senior Service Specialist,Fortune 500 2017-03-31,width: 20.0%;


1. creating columns for uncertain company ratings and bars

In [56]:
company_bar_score_names = ["Quick-paced", 'Slow-paced', 'Stressful', 'Balanced', 'Competitive', 'Relaxed', 'Cut-throat']
bar_dict = {'bar_name1' : 'bar_pct1', 
            'bar_name2' : 'bar_pct2', 
            'bar_name3' : 'bar_pct3', 
            'bar_name4' : 'bar_pct4', 
            'bar_name5' : 'bar_pct5'
}

company_rate_names = ["Compensation/Benefits", "Culture", "Job Security/Advancement", "Work/Life Balance", "Management"]
rate_dict = {'rating_name1' : 'rating_scr1', 
            'rating_name2' : 'rating_scr2', 
            'rating_name3' : 'rating_scr3', 
            'rating_name4' : 'rating_scr4', 
            'rating_name5' : 'rating_scr5'
}  

In [57]:
df['review_id'] = df.index + 127000
df['review_id'].head()

0    127000
1    127001
2    127002
3    127003
4    127004
Name: review_id, dtype: int64

Need to assign the appropriate rating columns and then match them to the new columns to be created in df

In [58]:
test = bar_dict.keys()
type(bar_dict)

dict

In [59]:
#testing for loop
test = df #need to add .copy() to test without changing df
i = 0
mask = test['bar_name2'] == company_bar_score_names[i]

test.loc[mask,company_bar_score_names[i]] = test.loc[mask,'bar_pct2']
print(company_bar_score_names[i])
test[company_bar_score_names[i]].head()

Quick-paced


0    44%
1    44%
2    44%
3    44%
4    44%
Name: Quick-paced, dtype: object

In [60]:
df.filter(regex = '^b').head()

,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,bar_pct1,bar_pct2,bar_pct3,bar_pct4,bar_pct5
0,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,20%
1,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,20%
2,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,20%
3,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,20%
4,Stressful,Quick-paced,Competitive,Cut-throat,Balanced,58%,44%,36%,27%,20%


In [61]:
#bar names
for name in company_bar_score_names:
    for bar, bar_pct in bar_dict.items():
        mask = df[bar].str.strip() == name
        df.loc[mask, name] = df.loc[mask, bar_pct]
        
    

In [62]:
#rating names
for name in company_rate_names:
    for rat, rat_scr in rate_dict.items():
        mask = df[rat].str.strip() == name
        df.loc[mask, name] = df.loc[mask, rat_scr]



In [63]:
df.iloc[:, -13:].head()

,review_id,Quick-paced,Slow-paced,Stressful,Balanced,Competitive,Relaxed,Cut-throat,Compensation/Benefits,Culture,Job Security/Advancement,Work/Life Balance,Management
0,127000,44%,NaN,58%,20%,36%,NaN,27%,3.5,3.2,3.1,3.1,3.0
1,127001,44%,NaN,58%,20%,36%,NaN,27%,3.5,3.2,3.1,3.1,3.0
2,127002,44%,NaN,58%,20%,36%,NaN,27%,3.5,3.2,3.1,3.1,3.0
3,127003,44%,NaN,58%,20%,36%,NaN,27%,3.5,3.2,3.1,3.1,3.0
4,127004,44%,NaN,58%,20%,36%,NaN,27%,3.5,3.2,3.1,3.1,3.0


In [64]:
df.iloc[:,-12:].isnull().apply(pd.value_counts)

,Quick-paced,Slow-paced,Stressful,Balanced,Competitive,Relaxed,Cut-throat,Compensation/Benefits,Culture,Job Security/Advancement,Work/Life Balance,Management
False,504257,260,504257,503837,495017,304800,208857,507557.0,507557.0,507557.0,507557.0,507557.0
True,3300,507297,3300,3720,12540,202757,298700,NaN,NaN,NaN,NaN,NaN


Create functions for converting width to percent and string percent to float

In [65]:
def string_pct_to_float(strpct):
    try:
        return float(strpct.strip('%'))/100
    except:
        return strpct

def parse_widths_to_rating(string):
    
    try:
        result = string_pct_to_float(string.replace("width: ","").replace(";","").strip())
        if result == 0:
            return np.nan
        else:
            return result * 5
    except:
        return string

In [66]:
print(string_pct_to_float('80%'))
print(string_pct_to_float('100%'))
print(string_pct_to_float(123))
print(parse_widths_to_rating('width: 100.0%'))
print(parse_widths_to_rating(123))
print(parse_widths_to_rating('width: 100.0%'))

0.8
1.0
123
5.0
123
5.0


In [67]:
#replace widths with 
widths_to_replace_mask = df.iloc[0,:].str.contains(r"^width: ") 

cols = df.columns[widths_to_replace_mask.fillna(False)]
cols

Index(['agg_rating', 'comp_ben_rating', 'culture_rating',
       'jobsec_advancement_rating', 'management_rating', 'work_life_rating'],
      dtype='object')

In [68]:
df[cols] = df[cols].applymap(parse_widths_to_rating)

agg_rating                   True
comp_ben_rating              True
culture_rating               True
jobsec_advancement_rating    True
management_rating            True
work_life_rating             True
Name: 0, dtype: object

In [69]:
df[cols].head()

,agg_rating,comp_ben_rating,culture_rating,jobsec_advancement_rating,management_rating,work_life_rating
0,5.0,5.0,5.0,5.0,5.0,5.0
1,4.0,4.0,4.0,4.0,3.0,3.0
2,5.0,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,5.0,5.0,5.0,5.0
4,3.0,3.0,4.0,4.0,2.0,1.0


In [70]:
bar_names_count = df[list(bar_dict.keys())].apply(pd.Series.value_counts)
bar_names_count['totals'] = bar_names_count.sum(axis = 1)
bar_names_count = bar_names_count.append(pd.Series(bar_names_count.sum(axis=0), name = 'subtotals'))
bar_names_count

,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,totals
Balanced,59880.0,102720.0,174419.0,147598.0,19220.0,503837.0
Competitive,24660.0,57700.0,160838.0,211719.0,40100.0,495017.0
Cut-throat,NaN,1280.0,11140.0,22420.0,174017.0,208857.0
Quick-paced,332297.0,125920.0,27920.0,17960.0,160.0,504257.0
Relaxed,800.0,880.0,19820.0,43280.0,240020.0,304800.0
Slow-paced,NaN,NaN,NaN,NaN,260.0,260.0
Stressful,86620.0,215757.0,110120.0,61280.0,30480.0,504257.0
subtotals,504257.0,504257.0,504257.0,504257.0,504257.0,2521285.0


In [71]:
val_cols = bar_names_count.index[:7]
#from above: df.filter(regex="^rat").isnull().apply(pd.value_counts)
val_col_counts = df[val_cols].isnull().apply(pd.value_counts)
val_col_counts['totals'] = val_col_counts.sum(axis = 1)
val_col_counts = val_col_counts.append(pd.Series(val_col_counts.sum(axis=0), name = 'subtotals'))
val_col_counts

,Balanced,Competitive,Cut-throat,Quick-paced,Relaxed,Slow-paced,Stressful,totals
False,503837,495017,208857,504257,304800,260,504257,2521285
True,3720,12540,298700,3300,202757,507297,3300,1031614
subtotals,507557,507557,507557,507557,507557,507557,507557,3552899


The values columns bnl check out with the others. I'll trust that the same goes for rating... it's late on Saturday and I don't have much time Sunday to work.

In [73]:
df.to_csv("indeed_scrapy/scrape_data/company_reviews100to500fort20171020" + "num_n_cowide sorted" + ".csv")